In [1]:
import folium
from IPython.display import display

# Create a map centered at a specific latitude and longitude
map_center = [0, 0]  # Change these coordinates to center the map as per your data
mymap = folium.Map(location=map_center, zoom_start=2)

# Add markers for each latitude and longitude data point
# Example data points
data_points = [(-10.154983333333332,-76.35935666666667)]

for lat, lon in data_points:
    folium.Marker(location=[lat, lon], popup=f"Latitude: {lat}, Longitude: {lon}").add_to(mymap)

# Display the map in the notebook
display(mymap)


In [2]:
import csv
locations_by_commodity = {}
def filter_locations(commodity_required, state_required, country_required, csv_file):
    filtered_locations = []
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            commodities = [c.strip() for c in row['commod1'].split(',')] + \
                          [c.strip() for c in row['commod2'].split(',')] + \
                          [c.strip() for c in row['commod3'].split(',')]
            if commodity_required in commodities:
                if (not state_required or row['state'] == state_required) and (not country_required or row['country'] == country_required):
                    filtered_locations.append({'latitude': row['latitude'], 'longitude': row['longitude']})
    return filtered_locations

def main():
    commodities_required = input("Enter the commodities required (comma-separated): ").split(',')
    state_required = input("Enter the state of interest (Press Enter to skip): ")
    country_required = input("Enter the country of interest (Press Enter to skip): ")
    
    csv_file = r'minerals.csv'  # Replace with your CSV file path
    
    
    for commodity_required in commodities_required:
        filtered_locations = filter_locations(commodity_required.strip(), state_required, country_required, csv_file)
        locations_by_commodity[commodity_required.strip()] = filtered_locations
    
    for commodity, locations in locations_by_commodity.items():
        if locations:
            print(f"\nLocations that have the required commodity '{commodity}':")
            for location in locations:
                print(f"Latitude: {location['latitude']}, Longitude: {location['longitude']}")
        else:
            print(f"\nNo locations found with the required commodity '{commodity}'.")


main()

Enter the commodities required (comma-separated): Copper
Enter the state of interest (Press Enter to skip): 
Enter the country of interest (Press Enter to skip): Peru

Locations that have the required commodity 'Copper':
Latitude: -6.83475, Longitude: -77.9931
Latitude: -9.49797, Longitude: -78.0831
Latitude: -9.66463, Longitude: -77.49976
Latitude: -8.24802, Longitude: -77.96644
Latitude: -9.96462, Longitude: -77.19975
Latitude: -8.13136, Longitude: -78.16644
Latitude: -8.19802, Longitude: -78.2331
Latitude: -8.13136, Longitude: -78.19977
Latitude: -9.78963, Longitude: -77.53309
Latitude: -9.73129, Longitude: -77.49976
Latitude: -9.74796, Longitude: -77.50254
Latitude: -9.82573, Longitude: -77.45864
Latitude: -9.83128, Longitude: -77.48309
Latitude: -9.81463, Longitude: -77.45808
Latitude: -9.94795, Longitude: -77.08307
Latitude: -9.07299, Longitude: -77.45808
Latitude: -9.4813, Longitude: -77.69976
Latitude: -10.44126, Longitude: -77.39975
Latitude: -10.03128, Longitude: -77.04974
La

In [3]:
import math
import folium
from IPython.display import display

def calculate_distance(coord1, coord2):
    """
    Calculate the Euclidean distance between two coordinates.
    """
    lat1, lon1 = float(coord1['latitude']), float(coord1['longitude'])
    lat2, lon2 = float(coord2['latitude']), float(coord2['longitude'])
    return math.sqrt((lat1 - lat2)**2 + (lon1 - lon2)**2)

def find_closest_coordinates(coord_set):
    """
    Find the pair of coordinates that are closest to each other in a given set of coordinates.
    """
    min_distance = float('inf')
    closest_coords = None
    
    # Iterate through all pairs of coordinates in the set
    for i in range(len(coord_set)):
        for j in range(i+1, len(coord_set)):
            distance = calculate_distance(coord_set[i], coord_set[j])
            if distance < min_distance:
                min_distance = distance
                closest_coords = (coord_set[i], coord_set[j])
    
    return closest_coords

def find_closest_coordinate_to_point(coord_set, point):
    """
    Find the coordinate in the set that is closest to a given point.
    """
    min_distance = float('inf')
    closest_coord = None
    
    # Iterate through all coordinates in the set
    for coord in coord_set:
        distance = calculate_distance(coord, point)
        if distance < min_distance:
            min_distance = distance
            closest_coord = coord
    
    return closest_coord

def find_closest_coordinates_in_dict(coord_dict):
    """
    Find the pair of coordinates that are closest to each other in each set of coordinates within a dictionary,
    and then select one coordinate from each set that is nearer to the previously selected coordinate.
    """
    closest_coords_dict = {}
    selected_coords = {}
    
    for key, value in coord_dict.items():
        closest_coords = find_closest_coordinates(value)
        closest_coord = closest_coords[0]  # Select the first coordinate in the closest pair
        selected_coords[key] = closest_coord
        
    for key, value in coord_dict.items():
        closest_coord = selected_coords[key]
        closest_coord_to_point = find_closest_coordinate_to_point(value, closest_coord)
        closest_coords_dict[key] = closest_coord_to_point
    
    return closest_coords_dict

def print_coordinates_on_map(coords_dict):
    """
    Print the coordinates on a map using Folium.
    """
    map_center = [0, 0]  # Change these coordinates to center the map as per your data
    mymap = folium.Map(location=map_center, zoom_start=2)
    
    for key, value in coords_dict.items():
        lat, lon = float(value['latitude']), float(value['longitude'])
        folium.Marker(location=[lat, lon], popup=f"{key} - Latitude: {lat}, Longitude: {lon}").add_to(mymap)
    
    display(mymap)

closest_coords_dict = find_closest_coordinates_in_dict(locations_by_commodity)
print("Selected coordinates from each set:")
for key, value in closest_coords_dict.items():
    print(f"{key}: {value}")

print("\nPrinting coordinates on map...")
print_coordinates_on_map(closest_coords_dict)


Selected coordinates from each set:
Copper: {'latitude': '-14.38056', 'longitude': '-73.06353'}

Printing coordinates on map...


In [4]:
#center coordinate
from IPython.display import display
def find_center_from_dict(coordinates_dict):
    if not coordinates_dict:
        return None
    
    num_points = len(coordinates_dict)
    sum_lat = sum(float(coord['latitude']) for coord in coordinates_dict.values())
    sum_lon = sum(float(coord['longitude']) for coord in coordinates_dict.values())
    
    center_lat = sum_lat / num_points
    center_lon = sum_lon / num_points
    
    return {'latitude': center_lat, 'longitude': center_lon}


center = find_center_from_dict(closest_coords_dict)
print("Center coordinate:", center)
map_center = folium.Map(location=[center['latitude'], center['longitude']], zoom_start=10)

# Add a marker at the center coordinate
folium.Marker(location=[center['latitude'], center['longitude']], popup='Center').add_to(map_center)
display(map_center)


Center coordinate: {'latitude': -14.38056, 'longitude': -73.06353}
